# Lesson 1 — Level 1 (Single cup)

Goal: build an **algorithm** (a sequence of moves) to drive around **one cup**.

You can use:
- `forward(seconds)`
- `backward(seconds)`
- `left(seconds)` / `right(seconds)`
- `turn_left(seconds)` / `turn_right(seconds)`
- `drift_left(seconds)` / `drift_right(seconds)`
- `horn()` (meep meep)

### Challenge
Write a plan that avoids the cup **without touching it**.

Tip: start slow. Then tighten it up.

In [ ]:
from lesson_header import *
print('Level 1 header loaded')


In [ ]:
from lesson_header import *
print('Robot ready')


## Example: a tiny test plan
This is just to prove things move. You’ll write your own plan below.

In [ ]:
from lesson_header import *

bot.horn()
bot.forward(0.5)
bot.stop()

## Challenge: Tune Drift Left (2 variables)
Use `bot.drift_left(seconds=..., turn_blend=...)`.

- `seconds`: how long the drift runs
- `turn_blend`: how curved the drift is (`0.0` = mostly sideways, `1.0` = strong turning arc)

Your challenge is to test both variables and find your best drift settings around a cup.


In [ ]:
from lesson_header import *

# Drift challenge: tune these two values
drift_seconds = 0.9
turn_blend = 0.55

print(f'Testing drift_left(seconds={drift_seconds}, turn_blend={turn_blend})')
bot.drift_left(seconds=drift_seconds, turn_blend=turn_blend)
bot.stop()


## Your plan (edit this)

Make a list of steps. Each step is ("move_name", seconds).

Moves you can use as strings:
`forward`, `backward`, `left`, `right`, `turn_left`, `turn_right`, `drift_left`, `drift_right`

Then run the plan.

In [ ]:
from lesson_header import *

# ============================
# YOUR PLAN (EDIT THIS)
# ============================

# Use these commands:

# bot.horn()
# bot.forward(0.5)
# bot.backward(0.5)
# bot.left(0.5)
# bot.right(0.5)
# bot.turn_left(0.5)
# bot.turn_right(0.5)
# bot.drift_left(0.5)
# bot.drift_right(0.5)
# bot.drift_right(0.5,0.55)  # drift with drift blend
# bot.stop()

# ----------------------------
# Write your moves below
# ----------------------------

bot.horn()

# Add your movement commands here


bot.stop()

print('Done')
